# RefSlave Agent Scenario

In [1]:
"""
LLM Helper Functions
"""

from typing import Dict, List, Optional
from pydantic import BaseModel
from datetime import datetime


def system_prompt() -> str:
    """현재 타임스탬프를 포함한 시스템 프롬프트를 생성합니다."""
    now = datetime.now().isoformat()
    return f"""당신은 전문 연구원입니다. 오늘 날짜는 {now}입니다. 응답 시 다음 지침을 따르세요:
    - 지식 컷오프 이후의 주제에 대한 조사를 요청받을 수 있습니다. 사용자가 뉴스 내용을 제시했다면, 그것을 사실로 가정하세요.
    - 사용자는 매우 숙련된 분석가이므로 내용을 단순화할 필요 없이 가능한 한 자세하고 정확하게 응답하세요.
    - 체계적으로 정보를 정리하세요.
    - 사용자가 생각하지 못한 해결책을 제안하세요.
    - 적극적으로 사용자의 필요를 예측하고 대응하세요.
    - 사용자를 모든 분야의 전문가로 대우하세요.
    - 실수는 신뢰를 저하시킵니다. 정확하고 철저하게 응답하세요.
    - 상세한 설명을 제공하세요. 사용자는 많은 정보를 받아들일 수 있습니다.
    - 권위보다 논리적 근거를 우선하세요. 출처 자체는 중요하지 않습니다.
    - 기존의 통념뿐만 아니라 최신 기술과 반대 의견도 고려하세요.
    - 높은 수준의 추측이나 예측을 포함할 수 있습니다. 단, 이를 명확히 표시하세요."""

## Report

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### Arxiv Loader


In [3]:
from langchain_community.document_loaders import ArxivLoader

loder = ArxivLoader(
    query="Chane of Thought",
    load_max_docs=10,
    load_all_available=True
)
docs = loder.load()

In [4]:
# json 데이터의 key값 확인
import json
data = docs[0].model_dump_json()
keys = json.loads(data).keys()
print("문서의 key 목록:", list(keys))
print("문서의 metadata:", docs[0].metadata)
print("문서의 id:", docs[0].id)

문서의 key 목록: ['id', 'metadata', 'page_content', 'type']
문서의 metadata: {'Published': '2017-01-17', 'Title': 'Slow-light analogue with a ladder of RLC circuits', 'Authors': 'J. -P. Cromières, T. Chanelière', 'Summary': 'The linear susceptibility of an atomic sample is formally equivalent to the\nresponse of a RLC circuit. We use a ladder of lumped RLC circuits to observe an\nanalogue of slow-light, a well-known phenomenon in atomic physics. We first\ncharacterize the radio-frequency response of the circuit in the spectral domain\nexhibiting a transparency window surrounded by two strongly absorptive lines.\nWe then observe a delayed pulse whose group delay is comparable to the pulse\nduration corresponding to slow-light propagation. The large group delay is\nobtained by cascading in a ladder configuration doubly resonant RLC cells.'}
문서의 id: None


In [5]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field


class SearchResult(BaseModel):
    title: str
    published: str
    summary: str
    authors: List[str]
    page_content: str


def parse_arxiv_result(doc) -> SearchResult:
    """Arxiv 문서를 SearchResult 객체로 변환하는 함수"""
    return SearchResult(
        title=doc.metadata.get('Title', ''),
        published=doc.metadata.get('Published', ''),
        summary=doc.metadata.get('Summary', ''),
        authors=doc.metadata.get('Authors', []).split(','),
        page_content=doc.page_content
    )


def search_arxiv(query: str, max_docs: int = 10) -> List[SearchResult]:
    """Arxiv 검색 결과를 SearchResult 객체 리스트로 반환하는 함수"""
    loader = ArxivLoader(
        query=query,
        load_max_docs=max_docs,
        load_all_available=True
    )
    docs = loader.load()
    return [parse_arxiv_result(doc) for doc in docs]

In [6]:
search_result = search_arxiv(query="Chain of Thought", max_docs=10)
search_result

[SearchResult(title='Contrastive Chain-of-Thought Prompting', published='2023-11-15', summary='Despite the success of chain of thought in enhancing language model\nreasoning, the underlying process remains less well understood. Although\nlogically sound reasoning appears inherently crucial for chain of thought,\nprior studies surprisingly reveal minimal impact when using invalid\ndemonstrations instead. Furthermore, the conventional chain of thought does not\ninform language models on what mistakes to avoid, which potentially leads to\nmore errors. Hence, inspired by how humans can learn from both positive and\nnegative examples, we propose contrastive chain of thought to enhance language\nmodel reasoning. Compared to the conventional chain of thought, our approach\nprovides both valid and invalid reasoning demonstrations, to guide the model to\nreason step-by-step while reducing reasoning mistakes. To improve\ngeneralization, we introduce an automatic method to construct contrastive\n

### SerpQuery

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


class SerpQuery(BaseModel):

    query: str
    research_goal: str


class SerpQueryResponse(BaseModel):
    queries: List[SerpQuery] = Field(
        ..., description="A list of SERP queries to be used for research"
    )


def generate_serp_queries(
    query: str,
    model_name: str,
    num_queries: int = 3,
    learnings: Optional[List[str]] = None,
) -> List[SerpQuery]:
    """
    사용자의 쿼리와 이전 연구 결과를 바탕으로 SERP 검색 쿼리를 생성합니다.
    json_parser를 사용해 구조화된 JSON을 반환
    """
    prompt = """
    ### System:
    {system_prompt}
    
    ### Instructions:
    다음 사용자의 입력을 기반으로 연구 주제를 조사하기 위한 SERP 검색 쿼리를 생성하세요.
    JSON 객체를 반환하며, 'queries' 배열 필드에 {num_queries}개의 쿼리를 포함합니다.(쿼리가 명확할 경우 더 적을 수도 있음)
    각 쿼리 객체에는 'query'와 'research_goal' 필드가 포함되어야 하며, 각 쿼리는 고유해야 합니다.
    
    ### Input:
    {query}
    """

    if learnings:  # 이전 연구에서 얻은 학습 내용이 있는 경우
        prompt += f"\n\n다음은 이전 연구에서 얻은 학습 내용입니다. 이를 활용하여 더 구체적인 쿼리를 생성하세요: {' '.join(learnings)}"

    prompt += "\n\n### Answer:\n\n{format_instructions}"

    prompt = PromptTemplate.from_template(prompt)

    json_parser = JsonOutputParser(pydantic_object=SerpQueryResponse)
    sys_prompt = system_prompt()

    prompt = prompt.partial(system_prompt=sys_prompt,
                            format_instructions=json_parser.get_format_instructions())

    llm = ChatGoogleGenerativeAI(model=model_name)

    chain = prompt | llm | json_parser
    response_json = chain.invoke({"query": query, "num_queries": num_queries})

    try:
        result = SerpQueryResponse.model_validate(response_json)
        queries = result.queries if result.queries else []
        print(f"리서치 주제에 대한 SERP 검색 쿼리 {len(queries)}개 생성 완료")
        return queries[:num_queries]
    except Exception as e:
        print(f"오류: generate_serp_queries에서 JSON 응답을 처리하는 중 오류 발생: {e}")
        print(f"원시 응답: {response_json}")
        print(f"오류: 쿼리 '{query}'에 대한 JSON 응답 처리 실패")
        return []

In [8]:
queries = generate_serp_queries(
    query="Chain of Thought의 최신 연구 동향", model_name="gemini-1.5-flash-8b", num_queries=3)

리서치 주제에 대한 SERP 검색 쿼리 3개 생성 완료


In [9]:
sample_query = queries[0].query

loder = ArxivLoader(
    query=sample_query,
    load_max_docs=10,
    load_all_available=True
)
search_result = loder.load()

In [10]:
# # print(docs[0].page_content)

# from markdownify import markdownify as md

# print(md(docs[0].page_content))

In [11]:
class ResearchResult(BaseModel):
    learnings: List[str]
    visited_urls: List[str]


class SerpResultResponse(BaseModel):
    learnings: List[str] = Field(description="검색 결과로부터 추출된 주요 학습 내용 목록")
    followUpQuestions: List[str] = Field(
        description="검색 결과를 바탕으로 생성된 후속 질문 목록")


def process_serp_result(
    query: str,
    search_result: List[SearchResult],
    model_name: str,
    num_learnings: int = 5,
    num_follow_up_questions: int = 3,
) -> Dict[str, List[str]]:
    """
    검색 결과를 처리하여 학습 내용과 후속 질문을 추출합니다.
    json_parser를 사용해 구조화된 JSON을 반환합니다.
    """
    contents = [item.page_content for item in search_result]

    json_parser = JsonOutputParser(pydantic_object=SerpResultResponse)
    contents_str = "".join(f"<내용>{content}</내용>" for content in contents)
    prompt = PromptTemplate(
        input_variables=["query", "num_learnings", "num_follow_up_questions"],
        template="""
        ### System:
        {system_prompt}
        
        ### Instruction:
        다음은 쿼리 <쿼리>{query}</쿼리>에 대한 SERP검색 결과입니다.
        이 내용을 바탕으로 학습 내용을 추출하고 후속 질문을 생성하세요.
        JSON 객체로 반환하며, 'learnings' 및 'followUpQuestions' 키를 포함한 배열을 반환하요.
        각 학습 내용은 고유하고 간결하며 정보가 풍부해야 합니다. 최대 {num_learnings}개의 학습 내용과
        {num_follow_up_questions}개의 후속 질문을 포함해야 합니다.\n\n
        <검색결과>{contents_str}</검색결과>
        
        ### Answer:
        {format_instructions}
        """
    ).partial(
        system_prompt=system_prompt(),
        contents_str=contents_str,
        format_instructions=json_parser.get_format_instructions()
    )
    llm = ChatGoogleGenerativeAI(model=model_name)
    chain = prompt | llm | json_parser
    response_json = chain.invoke({
        "query": query,
        "num_learnings": num_learnings,
        "num_follow_up_questions": num_follow_up_questions
    })

    try:
        result = SerpResultResponse.model_validate(response_json)
        return {
            "learnings": result.learnings,
            "followUpQuestions": result.followUpQuestions
        }
    except Exception as e:
        print(f"오류: process_serp_result에서 JSON 응답을 처리하는 중 오류 발생: {e}")
        print(f"원시 응답: {response_json}")
        return {
            "learnings": [],
            "followUpQuestions": []
        }


def deep_research(
    query: str,
    breadth: int,
    depth: int,
    model_name: str,
    learnings: Optional[List[str]] = None,
    visited_papers: Optional[List[str]] = None,
) -> ResearchResult:
    """
    주제를 재귀적으로 탐색하여 SERP 쿼리를 생성하고, 검색 결과를 처리하며,
    학습 내용과 방문한 URL을 수집합니다.
    """
    learnings = learnings or []
    visited_papers = visited_papers or []

    print(f"================Deep Research Start================\n")
    print(f"<주제> \n {query} \n <주제>")

    serp_queries = generate_serp_queries(
        query=query,
        model_name=model_name,
        num_queries=breadth,
        learnings=learnings
    )
    print(f"================SERP 쿼리 생성 완료================\n")
    print(f"{serp_queries}")

    for index, serp_query in enumerate(serp_queries, start=1):
        result: List[SearchResult] = search_arxiv(
            query=serp_query.query, max_docs=depth,)
        new_papers = [item.title for item in result if item.title]
        serp_result = process_serp_result(
            query=serp_query.query,
            search_result=result,
            model_name=model_name,
            num_learnings=5,
            num_follow_up_questions=breadth
        )

        all_learnings = learnings + serp_result['learnings']
        all_papers = new_papers + visited_papers
        new_depth = depth - 1
        new_breadth = max(1, breadth // 2)

        if new_depth > 0:
            nex_query = (
                f"이전 연구목표: {serp_query.research_goal}\n"
                f"후속 연구방향: {" ".join(serp_result['followUpQuestions'])}"
            )

            # 증가된 시도 획수로 재귀 호출
            sub_result = deep_research(
                query=nex_query,
                breadth=new_breadth,
                depth=new_depth,
                model_name=model_name,
                learnings=all_learnings,
                visited_papers=all_papers,
            )

            learnings = sub_result['learnings']
            visited_papers = sub_result['visited_papers']

        else:
            learnings = all_learnings
            visited_papers = all_papers

        return {
            "learnings": list(set(learnings)),
            "visited_papers": list(set(visited_papers))
        }

In [12]:
# result = process_serp_result(
#     query=sample_query,
#     search_result=search_result,
#     model_name="gemini-1.5-flash-8b",
#     num_learnings=5,
#     num_follow_up_questions=3
# )

In [13]:
# print(result['learnings'][0])

In [14]:
query = "RAG 연구 동향"
breadth = 3
depth = 3
model_name = "gemini-2.0-flash"
result = deep_research(query, breadth, depth, model_name)

================Deep Research Start================

<주제> 
 RAG 연구 동향 
 <주제>
리서치 주제에 대한 SERP 검색 쿼리 3개 생성 완료
================SERP 쿼리 생성 완료================

[SerpQuery(query='Recent advancements in Retrieval-Augmented Generation (RAG) models', research_goal='Identify the latest architectural innovations, training methodologies, and performance benchmarks for RAG models.'), SerpQuery(query='Challenges and limitations of current RAG implementations', research_goal='Investigate the common problems encountered when deploying RAG systems, such as knowledge staleness, hallucination, computational cost, and sensitivity to prompt engineering.'), SerpQuery(query='Applications of Retrieval-Augmented Generation (RAG) across different industries', research_goal='Explore diverse use cases of RAG in fields like healthcare, finance, education, and customer service, focusing on how RAG is tailored to specific domain requirements and data types.')]
================Deep Research Start================

<주제

In [15]:
result['learnings'][0]

'Adaptive Token-Layer Attention Scoring for Selective Retrieval (ATLAS)는 Multi-Layer Attention Gradient (MLAG)를 사용하여 모델 레이어 간의 주의 패턴 변화를 분석하여 정보 요구 사항을 효율적으로 결정합니다.'